In [1]:
import os
import json
import time
import threading
import unicodedata
import collections
import string
import re
import abc
import warnings
from typing import Optional, Dict, Tuple

import torch #spacy não carrega sem importar antes (??)
import spacy
import tqdm
import groq
import pyserini
from pyserini.search import SimpleSearcher
import sentence_transformers
import bs4

c:\Python38\lib\site-packages\thinc\compat.py:36: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  hasattr(torch, "has_mps")
c:\Python38\lib\site-packages\thinc\compat.py:37: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  and torch.has_mps  # type: ignore[attr-defined]


In [2]:
if not os.path.isdir("data"):
    os.mkdir("data")

if not os.path.isfile("data\\context_articles.json"):
    !curl -LO https://iirc-dataset.s3.us-west-2.amazonaws.com/context_articles.tar.gz
    !move context_articles.tar.gz data
    !tar -xf data/context_articles.tar.gz
    !move context_articles.json data

if not os.path.isfile("data\\iirc_test.json"):
    !curl -LO https://iirc-dataset.s3.us-west-2.amazonaws.com/iirc_test.json
    !move iirc_test.json data

In [3]:
file = open("data\\context_articles.json", "r")
articles = json.load(file)
file.close()

In [4]:
file = open("data\\iirc_test.json", "r")
test_data = json.load(file)
file.close()

In [5]:
n_question = 50

@fabiograssiotto: remoção de tags HTML

In [6]:
questions = []
documents_titles = set()
documents = []
documents_not_found = set()

for i in range(len(test_data)):
    item = test_data[i]

    name = item['title'].lower()
    if name not in documents_titles:
        text = item["text"]
        soup = bs4.BeautifulSoup(text, "html.parser")
        clean_text = soup.get_text()

        document = {
                    "title": item['title'],
                    "content": clean_text
                }
        
        documents.append(document)
        documents_titles |= {name}

    for link in item["links"]:
        name : str = link['target'].lower()
        if name in articles and name not in documents_titles:
            text = articles[link['target'].lower()]
            soup = bs4.BeautifulSoup(text, "html.parser")
            clean_text = soup.get_text()
            
            document = {
                "title": link['target'],
                "content": clean_text
            }

            documents.append(document)
            documents_titles |= {name}
        
        else:
            documents_not_found |= {name}

    for q in item["questions"]:
        question = {}
        

        if q["answer"]["type"] == "span":
            question["answer"] = q["answer"]["answer_spans"][0]["text"]
        elif q["answer"]["type"] == "value":
            question["answer"] = q["answer"]["answer_value"]+" "+q["answer"]["answer_unit"]
        elif  q["answer"]["type"] == "none":
            continue
        elif q["answer"]["type"] == "binary":
            question["answer"] = q["answer"]["answer_value"]
        else:
            raise ValueError
        
        question["question"] = q["question"]

        questions.append(question)

        if len(questions) == n_question:
            break
    if len(questions) == n_question:
            break


In [7]:
len(documents_not_found), len(documents)

(17, 419)

In [8]:
nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")

#12345
#   45678
stride = 3#2
max_length = 5#3

def window(documents, stride, max_length):
    treated_documents = []

    for j,document in enumerate(tqdm.tqdm(documents)):
        doc_text = document['content']
        doc = nlp(doc_text[:10000])
        sentences = [sent.text.strip() for sent in doc.sents]
        for i in range(0, len(sentences), stride):
            segment = ' '.join(sentences[i:i + max_length])
            treated_documents.append({
                "title": document['title'],
                "contents": document['title']+". "+segment,
                "segment": segment
            })
            if i + max_length >= len(sentences):
                break
    return treated_documents

treated_documents = window(documents, stride, max_length)

100%|██████████| 419/419 [00:03<00:00, 129.78it/s]


In [9]:
if not os.path.isdir("data\iirc_indices"):
    !mkdir data\iirc_indices

In [10]:
file = open("data/iirc_indices/contents.jsonl",'w')

for i, doc in enumerate(treated_documents):
    doc['id'] = i
    if doc['segment'] != "":
        file.write(json.dumps(doc)+"\n")

In [11]:
!python -m pyserini.index -collection JsonCollection -generator DefaultLuceneDocumentGenerator -threads 1 -input data/iirc_indices -index data/iirc_index -storeRaw

pyserini.index is deprecated, please use pyserini.index.lucene.
2024-05-14 15:05:18,277 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2024-05-14 15:05:18,280 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2024-05-14 15:05:18,280 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2024-05-14 15:05:18,280 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: data/iirc_indices
2024-05-14 15:05:18,280 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2024-05-14 15:05:18,281 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2024-05-14 15:05:18,281 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1
2024-05-14 15:05:18,281 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: en


In [12]:
index_path = "./data/iirc_index"

## Groq API

In [13]:
class GroqInterface:
    '''
    Interface for using the Groq API

    Implements a rate limit control for multi-threading use. 
    '''

    _client :groq.Groq = None 

    LLAMA3_70B = "llama3-70b-8192"

    inference_lock = threading.Lock()
    time_waiter_lock = threading.Lock()
    SINGLE_THREAD = True

    def __init__(self, model:Optional[str]=None, api_key:Optional[str]=None, json_mode:bool=False, system_message:str=None, n_retry:int=5):
        '''
        GroqInterface constructor.

        Args:
            model (str, optional): model to use. Llama3 70B is used if None. Default is None
        '''
        
        if GroqInterface._client is None:

            if api_key is None:
                api_key = os.environ.get("GROQ_API_KEY")

            if api_key is None:
                raise RuntimeError("API key is not in the environment variables ('GROQ_API_KEY' variable is not set).")

            GroqInterface._client = groq.Groq(api_key=api_key)

        if model is None:
            model = GroqInterface.LLAMA3_70B
        self._model = model

        self._system_message = system_message


        if json_mode:
            self._response_format = {"type": "json_object"}
        else:
            self._response_format = None
        self._json_mode = json_mode

        self._n_retry = n_retry

    def __call__(self, prompt:str) -> str:
        '''
        Generates the model response

        Args:
            prompt (str): prompt to send to the model.

        Returns:
            str: model response. 
        '''
        done = False
        retry_count = 0
        while not done:
            try:
                if not GroqInterface.SINGLE_THREAD:
                    GroqInterface.inference_lock.acquire()
                    GroqInterface.inference_lock.release()

                messages = []
                if self._system_message is not None:
                    messages.append({"role":"system", "content":self._system_message})
                
                messages.append({"role":"user", "content":prompt})

                chat_completion = GroqInterface._client.chat.completions.create(
                        messages=messages,
                        model=self._model,
                        response_format=self._response_format
                    )
                
                done = True
            except groq.RateLimitError as exception:
                print("ERROR")
                GroqInterface.error = exception
                if not GroqInterface.SINGLE_THREAD:
                    if not GroqInterface.time_waiter_lock.locked():
                        GroqInterface.time_waiter_lock.acquire()
                        GroqInterface.inference_lock.acquire()
                        time.sleep(2)
                        GroqInterface.time_waiter_lock.release()
                        GroqInterface.inference_lock.release()
                else:
                    time.sleep(2)
            except Exception as e:
                retry_count += 1
                if retry_count >= self._n_retry:
                    raise e

        return chat_completion.choices[0].message.content

In [14]:
groq_interface = GroqInterface()

In [15]:
groq_interface("Hi!")

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

## Metric

In [16]:
class GroqAnswerEvaluator(GroqInterface):

    _system_message = '''You are a answer comparator that outputs in JSON. 
The JSON object must use the schema: {'is_correct':'bool'}
Please use a valid JSON format.'''

    _base_prompt = '''Is the second answer correct, comparing to the first one? 
First answer: {answer_correct}
Second answer: {answer} 
'''

    def __init__(self, model: Optional[str] = None, api_key: Optional[str]=None):
        super().__init__(model, api_key, True, GroqAnswerEvaluator._system_message)

    def __call__(self, answer_correct:str, answer:str) -> int:

        prompt = GroqAnswerEvaluator._base_prompt.format(answer_correct=answer_correct, answer=answer)

        response = super().__call__(prompt)
        response = json.loads(response)

        return response["is_correct"]

In [17]:
a_gold = "sky and thunder god"
a_correct = "Zeus is known as the king of the gods and the god of the sky and thunder in Greek mythology."
a_wrong = "Zeus is been depicted as using violence to get his way and terrorize humans."

In [18]:
groq_corrector = GroqAnswerEvaluator()

In [19]:
groq_corrector(a_gold, a_correct), groq_corrector(a_gold, a_wrong),

(True, False)

In [20]:
def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  def remove_accents(input_str):
      nfkd_form = unicodedata.normalize('NFKD', input_str)
      only_ascii = nfkd_form.encode('ASCII', 'ignore')
      return only_ascii.decode("utf-8")

  return white_space_fix(remove_articles(remove_punc(lower(remove_accents(s)))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
  return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

In [21]:
compute_f1(a_gold, a_correct), compute_exact(a_gold, a_correct)

(0.4, 0)

In [22]:
compute_f1(a_gold, a_wrong), compute_exact(a_gold, a_wrong)

(0.11111111111111112, 0)

## ReAct implementation

- classe que implementa tool
- Tools: think, search, answer.
- Entrada e saída de todas as tools bem definidas
- Contexto tem apenas saída das tools

- Limite de etapas, depois de n obriga a próxima a ser answer

### Tools

In [23]:
class Tool(abc.ABC):

    @abc.abstractmethod
    def __call__(self, query:str, context:str):
        ...

In [24]:
class SearchTool(Tool):

    def __init__(self, index_path:str, embedder_model:str="all-MiniLM-L6-v2", search_k:int=20, rerank_k:int=5):
        self.embedder = sentence_transformers.SentenceTransformer(embedder_model)
        self.searcher = SimpleSearcher(index_path)

        if search_k < rerank_k:
            warnings.warn(f"search_k is less than rerank_k. The result will be of search_k size. ({search_k} < {rerank_k})")
        
        self.search_k = search_k
        self.rerank_k = rerank_k
    
    def __call__(self, query:str, context=None) -> Dict[str, str]:
        query_embedding = self.embedder.encode(query, convert_to_tensor=True)

        search_result = self.searcher.search(query, k=self.search_k)
        search_docs = []
        for result in search_result:
            result = json.loads(result.raw)

            search_docs.append(result["segment"])

        search_embeddings = self.embedder.encode(search_docs, convert_to_tensor=True)


        rerank_result = sentence_transformers.util.semantic_search(query_embedding, search_embeddings, top_k=self.rerank_k)

        response = ""
        for result in rerank_result[0]:
            index = result["corpus_id"]
            result_doc = search_docs[index]

            response += result_doc + "\n"
        
        response = {"result":response}


        return response

In [25]:
class AnswerTool(Tool, GroqInterface):
    _system_message = '''You are a question answerer that outputs in JSON. 
The JSON object must use the schema: {'answer':'str'}
Please use a valid JSON format.'''

    _base_prompt = '''{context}

Considering all this context, provide the FINAL ANSWER to the question:
'''

    def __init__(self, model: Optional[str] = None, api_key: Optional[str] = None):
        super().__init__(model, api_key, True, AnswerTool._system_message)

    def __call__(self, query: Optional[str]=None, context: str=None):
        prompt = AnswerTool._base_prompt.format(context=context)

        return json.loads(GroqInterface.__call__(self, prompt=prompt))

In [26]:
class ThinkTool(Tool, GroqInterface):
    _system_message = '''You are responsible for reflecting on provided information, and outputs in JSON. 
The JSON object must use the schema: {'reflection':'str'}
Please use a valid JSON format.'''

    _base_prompt = '''{context}

Considering all this context, provide a useful reflection.
'''

    def __init__(self, model: Optional[str] = None, api_key: Optional[str] = None):
        super().__init__(model, api_key, True, ThinkTool._system_message)

    def __call__(self, query: Optional[str]=None, context: str=None):
        prompt = ThinkTool._base_prompt.format(context=context)

        return json.loads(GroqInterface.__call__(self, prompt=prompt))    

In [27]:
class RoutingTool(Tool, GroqInterface):
    _system_message = '''You are a tool selector that outputs in JSON.
For each selection, you must provide a throught about the previous content and why you are selecting this tool.

You have available the following tools:

Name: 'search'
Query content: the sentence to search
Tool description: it searchs for documents with possible usefull information using the query.

Name: 'answer'
Query content: empty (''), is not used.
Tool description: considering the previous content, provides the final answer for the question.

Name: 'think'
Query content: empty (''), is not used;
Tool descripting: considering the previous content, generates a useful reflection.

The JSON object must use the schema: {'throught':'str',
'tool_name':'str',
'query':'str'}

Please use a valid JSON format.'''

    _base_prompt = '''{context}
    
Considering all this context, select the next tool to use:
    '''

    def __init__(self, model: Optional[str] = None, api_key: Optional[str] = None):
        super().__init__(model, api_key, True, RoutingTool._system_message)

    def __call__(self, query=None, context=None):
        prompt = RoutingTool._base_prompt.format(context=context)

        

        return json.loads(GroqInterface.__call__(self, prompt=prompt))

In [28]:
search_tool = SearchTool(index_path)
answer_tool = AnswerTool()
think_tool = ThinkTool()

routing_tool = RoutingTool()

SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead


In [29]:
test_question = "What is Zeus know for in Greek mythology?"

In [30]:
test_context = f'''Question: {test_question}'''

routing_tool(context=test_context)

{'thought': 'The user has asked a question about Greek mythology, I need to find relevant information.',
 'tool_name': 'search',
 'query': 'What is Zeus known for in Greek mythology?'}

In [31]:
search_result = search_tool("Zeus Greek mythology", "")

In [43]:
print(search_result["result"])

Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion, who rules as king of the gods of Mount Olympus. His name is cognate with the first element of his Roman equivalent Jupiter. His mythologies and powers are similar, though not identical, to those of Indo-European deities such as Jupiter, Perkūnas, Perun, Indra and Thor. Zeus is the child of Cronus and Rhea, the youngest of his siblings to be born, though sometimes reckoned the eldest as the others required disgorging from Cronus's stomach. In most traditions, he is married to Hera, by whom he is usually said to have fathered Ares, Hebe, and Hephaestus.
Since lightning is quite powerful and sometimes deadly, it is a bold sign when lightning strikes because it is known that Zeus most likely threw the bolt. In the Iliad. The Iliad is a poem by Homer about the Trojan war and the battle over the City of Troy, in which Zeus plays a major part. Scenes in which Zeus appears include:



In [32]:
test_context2 = f'''
Question: {test_question}

Thought: I need to search for information about Zeus in Greek mythology to find the answer.
Search result: {search_result["result"]}
'''

routing_tool(context=test_context2)

{'thought': 'Now that I have searched and found information about Zeus in Greek mythology, I can reflect on the key points to understand the essence of his character and significance.',
 'tool_name': 'think',
 'query': ''}

In [33]:
test_context3 = f'''
Question: {test_question}

Thought: I need to search for information about Zeus in Greek mythology to find the answer.
Search result: {search_result["result"]}

Thought: I have gathered information about Zeus in Greek mythology, now I need to think about the context to understand the main points and reflect on the answer.
'''

think_tool(context=test_context3)

{'reflection': "Zeus, in Greek mythology, is the king of the gods, known for being the sky and thunder god, ruling from Mount Olympus. He wields significant power, controlling the weather and fate of humans, often using his lightning bolts to intervene in mortal affairs. Throughout literature, such as in Homer's Iliad, Zeus plays a major role, influencing the war and its outcome. His character is multifaceted, displaying emotions, making decisions, and interacting with other gods and mortals. His power and influence are undeniable, making him a central figure in Greek mythology."}

In [34]:
test_context4 = f'''
Question: {test_question}

Thought: I need to search for information about Zeus in Greek mythology to find the answer.
Search result: {search_result["result"]}

Thought: I have gathered information about Zeus in Greek mythology, now I need to think about the context to understand the main points and reflect on the answer.
Think result: Zeus is known as the sky and thunder god in ancient Greek religion and is often depicted holding a thunderbolt. He is the king of the gods and is married to Hera, with whom he has fathered several children. His powers and mythologies are similar to those of other Indo-European deities, such as Jupiter in Roman mythology. He is often depicted in art standing or seated in majesty, holding a scepter, and is associated with the eagle, bull, and oak."
'''

routing_tool(context=test_context4)

{'thought': 'I have thought about the context and I think it is time to answer the question.',
 'tool_name': 'answer',
 'query': ''}

In [35]:
test_context5 = f'''
Question: {test_question}

Thought: I need to search for information about Zeus in Greek mythology to find the answer.
Search result: {search_result["result"]}

Thought: I have gathered information about Zeus in Greek mythology, now I need to think about the context to understand the main points and reflect on the answer.
Think result: Zeus is known as the sky and thunder god in ancient Greek religion, and is the king of the gods of Mount Olympus.

Thought: "I\'ve reflected on the information and main points about Zeus, now I need to provide a concise answer to the original question."
'''

answer_tool(context=test_context5)

{'answer': 'Zeus is known as the sky and thunder god in ancient Greek religion, and is the king of the gods of Mount Olympus.'}

### Main code

In [36]:
class QuestionAnswerer:

    def __init__(self, search_tool:SearchTool, routing_tool:RoutingTool=None, think_tool:ThinkTool=None, answer_tool:AnswerTool=None ) -> None:
        self._search_tool = search_tool

        if routing_tool is None:
            routing_tool = RoutingTool()

        if think_tool is None:
            think_tool = ThinkTool()

        if answer_tool is None:
            answer_tool = AnswerTool()

        self._routing_tool = routing_tool
        self._think_tool = think_tool
        self._answer_tool = answer_tool

        
    def __call__(self, question:str, max_steps:int = 5, verbose=False) -> Tuple[str, str]:
        context = f"Question: {question}"

        step = 0

        while True:
            
            if step >= max_steps-1:
                next_tool = "answer"
                query = ""
                thought = "Now I need to provide a concise answer to the original question."
            else:
                if verbose:
                    print("R", end="")

                rounting_result = self._routing_tool(context=context)
                next_tool = rounting_result["tool_name"].lower()
                query = rounting_result["query"]
                thought = rounting_result["thought"]

            context += f"\n\nThought: {thought}\n"

            if verbose:
                print(next_tool[0].upper(), end="")

            try:

                if next_tool == "search":
                    search_result = self._search_tool(query=query, context=context)
                    context += f"Search result: {search_result['result']}"
                elif next_tool == "think":
                    think_result = self._think_tool(query=query, context=context)
                    context += f"Think result: {think_result['reflection']}"
                elif next_tool == "answer":
                    answer_result = self._answer_tool(query=query, context=context)

                    print("")
                    return answer_result["answer"], context
            except:
                print("F", end="")
                context += "The tool failed. I can try using it again, or try using another tool."
                
            
            step += 1

In [37]:
question_answerer = QuestionAnswerer(search_tool)

In [39]:
answer, context = question_answerer(test_question, verbose=True)

RSRTRA


In [40]:
answer

'Zeus is known as the king of the gods, god of the sky and thunder, and is famous for his powerful lightning bolts in Greek mythology.'

In [41]:
print(context)

Question: What is Zeus know for in Greek mythology?

Thought: To get started, I need to search for information related to Zeus in Greek mythology.
Search result: Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion, who rules as king of the gods of Mount Olympus. His name is cognate with the first element of his Roman equivalent Jupiter. His mythologies and powers are similar, though not identical, to those of Indo-European deities such as Jupiter, Perkūnas, Perun, Indra and Thor. Zeus is the child of Cronus and Rhea, the youngest of his siblings to be born, though sometimes reckoned the eldest as the others required disgorging from Cronus's stomach. In most traditions, he is married to Hera, by whom he is usually said to have fathered Ares, Hebe, and Hephaestus.
Since lightning is quite powerful and sometimes deadly, it is a bold sign when lightning strikes because it is known that Zeus most likely threw the bolt. In the Iliad.

In [44]:
groq_corrector(a_gold, answer)

True